In [1]:
#load libraries
from pandasticsearch.client import RestClient
from pandasticsearch.queries import Agg, Select
from pandasticsearch.operators import *
from pandasticsearch.types import Column
from pandasticsearch.types import Row
from elasticsearch import Elasticsearch
from pandasticsearch import Select
from pandasticsearch import DataFrame
from pandas import DataFrame
import numpy as np
import pandas as pd
import json
from os import environ
from espandas import Espandas
from pandas import DataFrame, Series
import sqlite3 as db
import requests
import json
import json, time
from pandas.io.json import json_normalize
import os.path

In [2]:
# declare globals for the Elasticsearch client host
DOMAIN = "localhost"
PORT = 9200
es_client = Elasticsearch() # localhost

In [3]:
try:
# use the JSON library's dump() method for indentation
    info = json.dumps(es_client.info(), indent=4)

# pass client object to info() method
    print ("Elasticsearch client info():", info)

except exceptions.ConnectionError as err:

# print ConnectionError for Elasticsearch
    print ("nElasticsearch info() ERROR:", err)
    print ("nThe client host:", host, "is invalid or cluster is not running")

# change the client's value to 'None' if ConnectionError
    es_client = None

Elasticsearch client info(): {
    "name": "PC13409",
    "cluster_name": "elasticsearch",
    "cluster_uuid": "UrHsFJWCRwKapNIU8y7QKQ",
    "version": {
        "number": "7.4.2",
        "build_flavor": "default",
        "build_type": "zip",
        "build_hash": "2f90bbf7b93631e52bafb59b3b049cb44ec25e96",
        "build_date": "2019-10-28T20:40:44.881551Z",
        "build_snapshot": false,
        "lucene_version": "8.2.0",
        "minimum_wire_compatibility_version": "6.8.0",
        "minimum_index_compatibility_version": "6.0.0-beta1"
    },
    "tagline": "You Know, for Search"
}


In [4]:
#connection to elasticsearch (localhost). two indeces connection with scroll time 1 minute
elastic_url_1 = 'http://localhost:9200/cdr_1/_search?scroll=1m'
elastic_url_2 = 'http://localhost:9200/cdr_2/_search?scroll=1m'
elastic_url_3 = 'http://localhost:9200/cdr_2/_search?scroll=1m'


#scroll_api_url = 'http://localhost:9200/_search/scroll'
headers = {'Content-Type': 'application/json'}

In [5]:
#filter search in DSL Elasticsearch

# select data with countrycode = 0
#payload = {
#    "size": 100,
#    "query": {
#        "match" : {
#            "countrycode" : "0"
#        }
#    }
#}

# select all data from ES index with batch size 100
payload = {
        "size": 100,
        "query": {
            "match_all": {}
        }
    }

# select data with internet <> Null
#payload = {"size": 100,"query": {"constant_score": {"filter": {"exists": {"field": "internet"}}}}}


In [6]:
##Data retrive for 1st Index##

In [7]:
r1 = requests.request(
    "POST",
    elastic_url_1,
    data=json.dumps({"size": 100,"query": {"constant_score": {"filter": {"exists": {"field": "internet"}}}}}),
    headers=headers)

In [8]:
print(r1)

<Response [200]>


In [27]:
# first batch data for 1st index
try:
    res_json = r1.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r1.json()))

In [28]:
print (data)

[{'_index': 'cdr_1', '_type': '_doc', '_id': 'ZCPBt24BIq6EnoN8H03q', '_score': 1.0, '_source': {'datetime': '2013-11-01 01:00:00', '@timestamp': '2013-11-01T01:00:00.000+02:00', 'countrycode': 43, 'CellID': 7634, 'internet': 0.2433}}, {'_index': 'cdr_1', '_type': '_doc', '_id': 'ZiPBt24BIq6EnoN8H03q', '_score': 1.0, '_source': {'datetime': '2013-11-01 01:00:00', '@timestamp': '2013-11-01T01:00:00.000+02:00', 'countrycode': 31, 'CellID': 7635, 'internet': 0.0178}}, {'_index': 'cdr_1', '_type': '_doc', '_id': 'ZyPBt24BIq6EnoN8H03q', '_score': 1.0, '_source': {'callout': 2.8076, 'smsout': 1.6522, 'countrycode': 39, 'CellID': 7635, 'datetime': '2013-11-01 01:00:00', '@timestamp': '2013-11-01T01:00:00.000+02:00', 'smsin': 9.5432, 'callin': 0.1807, 'internet': 190.4214}}, {'_index': 'cdr_1', '_type': '_doc', '_id': 'aCPBt24BIq6EnoN8H03q', '_score': 1.0, '_source': {'datetime': '2013-11-01 01:00:00', '@timestamp': '2013-11-01T01:00:00.000+02:00', 'countrycode': 43, 'CellID': 7635, 'internet':

In [29]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({'scroll': '2m','_scroll_id': _scroll_id})
    print(scroll_payload)
        
    scroll_res = requests.request(
    "POST", elastic_url_1,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    #reform a specific dataset 
    #dataset = DataFrame(result_dict_1,columns=['_source.@timestamp','_source.CellID','_source.callin','_source.callout',
    #                                      '_source.countrycode','_source.datetime','_source.smsin','_source.smsout',
    #                                       '_source.internet'])
    
    dataset = DataFrame(result_dict_1,columns=['_id','_source.@timestamp','_source.CellID','_source.countrycode',
                                               '_source.datetime','_source.internet'])
    
    #rename columns
    #dataset.columns=['@timestamp','CellID','callin','callout','countrycode','datetime',
    #                 'smsin','smsout','internet']
    
    dataset.columns=['id','@timestamp','CellID','countrycode','datetime','internet']
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="cdr_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

{"scroll": "2m", "_scroll_id": "DXF1ZXJ5QW5kRmV0Y2gBAAAAAAAANoAWY2FzVE96d1FTcy10M3RKTFVaOURxZw=="}
Error: Elastic Search Scroll: {'error': {'root_cause': [{'type': 'parsing_exception', 'reason': 'Unknown key for a VALUE_STRING in [scroll].', 'line': 1, 'col': 12}], 'type': 'parsing_exception', 'reason': 'Unknown key for a VALUE_STRING in [scroll].', 'line': 1, 'col': 12}, 'status': 400}


In [12]:
##Data retrive for 2nd Index##

In [13]:
r2 = requests.request(
    "POST",
    elastic_url_2,
    data=json.dumps(payload),
    headers=headers)

In [14]:
# first batch data for 2nd index
try:
    res_json = r2.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r2.json()))

In [15]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({
    'scroll': '1m',
    'scroll_id': _scroll_id
    })
        
    scroll_res = requests.request(
    "POST", scroll_api_url,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    #reform a specific dataset 
    #dataset = DataFrame(result_dict_1,columns=['_source.@timestamp','_source.CellID','_source.callin','_source.callout',
    #                                      '_source.countrycode','_source.datetime','_source.smsin','_source.smsout',
    #                                       '_source.internet'])
    
    dataset = DataFrame(result_dict_1,columns=['_id','_source.@timestamp','_source.CellID','_source.countrycode',
                                               '_source.datetime','_source.internet'])
    
    #rename columns
    #dataset.columns=['@timestamp','CellID','callin','callout','countrycode','datetime',
    #                 'smsin','smsout','internet']
    
    dataset.columns=['id','@timestamp','CellID','countrycode','datetime','internet']
    
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="cdr_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

NameError: name 'scroll_api_url' is not defined

In [ ]:
##Data retrive for 3rd Index##

In [ ]:
r3 = requests.request(
    "POST",
    elastic_url_3,
    data=json.dumps(payload),
    headers=headers)

In [ ]:
# first batch data for 3rd index
try:
    res_json = r3.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r3.json()))

In [ ]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({
    'scroll': '1m',
    'scroll_id': _scroll_id
    })
        
    scroll_res = requests.request(
    "POST", scroll_api_url,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    #reform a specific dataset 
    #dataset = DataFrame(result_dict_1,columns=['_source.@timestamp','_source.CellID','_source.callin','_source.callout',
    #                                      '_source.countrycode','_source.datetime','_source.smsin','_source.smsout',
    #                                       '_source.internet'])
    
    dataset = DataFrame(result_dict_1,columns=['_id','_source.@timestamp','_source.CellID','_source.countrycode',
                                               '_source.datetime','_source.internet'])
    
    #rename columns
    #dataset.columns=['@timestamp','CellID','callin','callout','countrycode','datetime',
    #                 'smsin','smsout','internet']
    
    dataset.columns=['id','@timestamp','CellID','countrycode','datetime','internet']
    
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="cdr_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

In [ ]:
################################################################################################################################

In [ ]:
#connection to elasticsearch (localhost). two indeces connection with scroll time 1 minute
elastic_url_4 = 'http://localhost:9200/mi_1/_search?scroll=1m'
elastic_url_5 = 'http://localhost:9200/mi_2/_search?scroll=1m'
elastic_url_6 = 'http://localhost:9200/mi_3/_search?scroll=1m'

scroll_api_url = 'http://localhost:9200/_search/scroll'
headers = {'Content-Type': 'application/json'}

In [ ]:
# select data with provinceName <> Null
payload = {    
"size": 100,
     "query" : {
      "constant_score" : {
         "filter" : {
            "exists" : {
               "field" : "provinceName"
            }
         }
      }
   }
}

In [ ]:
##Data retrive for 1st mi Index##

In [ ]:
r4 = requests.request(
    "POST",
    elastic_url_4,
    data=json.dumps(payload),
    headers=headers)

In [ ]:
# first batch data for 1st mi Index
try:
    res_json = r4.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r4.json()))

In [ ]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({
    'scroll': '1m',
    'scroll_id': _scroll_id
    })
        
    scroll_res = requests.request(
    "POST", scroll_api_url,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    
    dataset = DataFrame(result_dict_1,columns=['_source.datetime','_source.@timestamp','_source.CellID','_source.provinceName',
                                               '_id'])
    
    
    dataset.columns=['datetime','@timestamp','CellID','provinceName','id']
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="mi_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

In [ ]:
##Data retrive for 2nd mi Index##

In [ ]:
r5 = requests.request(
    "POST",
    elastic_url_5,
    data=json.dumps(payload),
    headers=headers)

In [ ]:
# first batch data for 1st mi Index
try:
    res_json = r5.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r5.json()))

In [ ]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({
    'scroll': '1m',
    'scroll_id': _scroll_id
    })
        
    scroll_res = requests.request(
    "POST", scroll_api_url,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    
    dataset = DataFrame(result_dict_1,columns=['_source.datetime','_source.@timestamp','_source.CellID','_source.provinceName',
                                               '_id'])
    
    
    dataset.columns=['datetime','@timestamp','CellID','provinceName','id']
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="mi_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

In [ ]:
##Data retrive for 3rd mi Index##

In [ ]:
r6 = requests.request(
    "POST",
    elastic_url_6,
    data=json.dumps(payload),
    headers=headers)

In [ ]:
# first batch data for 1st mi Index
try:
    res_json = r6.json()
    data = res_json['hits']['hits']
    _scroll_id = res_json['_scroll_id']

except KeyError:
    data = []
    _scroll_id = None
    print ('Error: Elastic Search: %s' % str(r6.json()))

In [ ]:
#loop and retrive data from ES and store them in an SQLite DB
while data:

    #print (data)
      
    # scroll to get next batch data
    scroll_payload = json.dumps({
    'scroll': '1m',
    'scroll_id': _scroll_id
    })
        
    scroll_res = requests.request(
    "POST", scroll_api_url,
    data=scroll_payload,
    headers=headers
    )
    
    #normalize data from raw json to columns
    result_dict_1 = json_normalize(data)
    
    
    dataset = DataFrame(result_dict_1,columns=['_source.datetime','_source.@timestamp','_source.CellID','_source.provinceName',
                                               '_id'])
    
    
    dataset.columns=['datetime','@timestamp','CellID','provinceName','id']
    
    #print(dataset)
    
    #connect to sqlite database
    connex = db.connect("store.db") 
    cur = connex.cursor() 
    
    #export dataset to sqlite
    dataset.to_sql(name="mi_dataset", con=connex, if_exists="append", index=False)   

    #scroll data with unique id (avoid duplicates)           
    try:
        res_json = scroll_res.json()
        data = res_json['hits']['hits']
        _scroll_id = res_json['_scroll_id']

    except KeyError:
        data = []
        _scroll_id = None
        err_msg = 'Error: Elastic Search Scroll: %s'
        print (err_msg % str(scroll_res.json()))

In [ ]:
#######################################################################################################################

In [ ]:
#sqlite DB connection
connex = db.connect("store.db") 
cur = connex.cursor()

In [ ]:
#query SQL selections

#query = "SELECT * FROM cdr_dataset;"
#query = "SELECT * FROM cdr_dataset WHERE CellID=3387 AND internet>=100;"
#query = "SELECT * FROM cdr_dataset WHERE countrycode=39 AND internet>=10;"
#query = "SELECT datetime, CellID, internet FROM cdr_dataset WHERE internet>=100;"
#query = "SELECT CellID, callin, callout, countrycode, datetime, smsin, smsout, internet FROM cdr_dataset;"

query = "SELECT DISTINCT * FROM cdr_dataset as cdr, mi_dataset as mi WHERE cdr.datetime=mi.datetime AND cdr.CellID=mi.CellID AND cdr.internet>=1;"

#query = "SELECT DISTINCT cdr.id, cdr.@timestamp,cdr.CellID,cdr.countrycode,cdr.datetime,mi.provinceName,cdr.internet FROM cdr_dataset as cdr, mi_dataset as mi WHERE cdr.datetime=mi.datetime AND cdr.CellID=mi.CellID AND cdr.internet>=1;"


In [ ]:
df = pd.read_sql_query(query,connex)


In [ ]:
df.head()
#print(df)

In [ ]:
df2 = df.drop(df.columns[[6, 7, 8, 10]], axis=1)

In [ ]:
print(df2)

In [ ]:
# fill with 0 where is NaN
#pandas_df = df.fillna(0)
#pandas_df
#pandas_df.head()

In [ ]:
# see the columns of our dataset
#pandas_df.columns

In [ ]:
#select columns from pandas
#countrycode = DataFrame(pandas_df,columns=['countrycode'])
#countrycode

#callin =  DataFrame(pandas_df,columns=['callin'])
#callin

In [ ]:
# making aggregations and conditions in pandas
#pandas_df['callout_pricing'] = pandas_df.apply (lambda x: x['callout']*0.25 if x['countrycode']==221 else x['callout']*0.18, axis=1)
#pandas_df['callin_pricing'] = pandas_df.apply (lambda x: x['callin']*0.019 if x['countrycode']==221 else x['callin']*0.011, axis=1) 
#pandas_df['smsin_pricing'] = pandas_df.apply (lambda x: x['smsin']*0.01 if x['countrycode']==221 else x['smsin']*0.01, axis=1) 
#pandas_df['smsout_pricing'] = pandas_df.apply (lambda x: x['smsout']*0.15 if x['countrycode']==221 else x['smsout']*0.1, axis=1) 
#pandas_df['internet_pricing'] = pandas_df.apply (lambda x: x['internet']*0.7 if x['countrycode']==221 else x['internet']*0.4, axis=1) 


In [ ]:
#print(pandas_df)
#pandas_df.head()

In [ ]:
# summurize all the column values in one total column
#pandas_df['total_pricing'] = pandas_df.apply (lambda x: x['callout_pricing'] + x['callin_pricing'] + x['smsin_pricing'] + x['smsout_pricing'] + x['internet_pricing'] , axis=1)


In [ ]:
#print(pandas_df)
#pandas_df.head()

In [ ]:
# add one more column in the dataset (index column in this case)
#pandas_df['uid_name'] = (pandas_df.index + 0).astype(str)
df2['uid_name'] = (df2.index + 0).astype(str)

In [ ]:
pandas_df=df2
#print(df)
pandas_df.head()

In [ ]:
#pandas_df.head()

In [ ]:
# Export the final datast in csv file
# Don't forget to add '.csv' at the end of the path
export_csv = pandas_df.to_csv("C:/Users/LeliopoulosP/Desktop/ElasticPython/final data retrive from ES and SQLite storaging/export.csv", index = None, header=True)

In [ ]:
#csv_dataset = pd.read_csv("export.csv", index_col ="id") 

In [ ]:
# Convert a panda's dataframe to json
df = pandas_df.read_csv("export.csv",sep=",",decimal=".")

In [ ]:
# Add an id for looping into elastic search index
df["no_index"] = [x+1 for x in range(len(df["datetime"]))]

In [ ]:
# Convert into json
tmp = df.to_json(orient = "records")

In [ ]:
# Load each record into json format before bulk
df_json= json.loads(tmp)
df_json[:10]

In [ ]:
# creating an index in the Elasticsearch
INDEX = 'cdr_mi'
TYPE = 'bar_type'
esp = Espandas()
esp.es_write(df, INDEX, TYPE)

In [ ]:
# parce the dataset in the elasticsearch
def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

In [ ]:
e = Elasticsearch() # no args, connect to localhost:9200
if not e.indices.exists(INDEX):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX)

In [ ]:
r = e.bulk(rec_to_actions(df)) # return a dict
print(not r["errors"])

In [30]:
##delete sqlite file
connex.close()
##os.remove("store.db")